<a href="https://colab.research.google.com/github/ZeBread/tester/blob/main/trail_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#PACKAGES
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
#DRIVE LINKING
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
#DATA DIRECTORY LINKING
DATADIR = "/content/drive/MyDrive/train_200px"
CATEGORIES = ["FH","MC","MH","NL","OH","VC"]

CREATING DATASET

In [4]:
batch_size = 5
img_height = 200
img_width = 200

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATADIR,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 592 files belonging to 6 classes.
Using 533 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATADIR,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 592 files belonging to 6 classes.
Using 59 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['FH_200px', 'MC_200px', 'MH_200px', 'NL_200px', 'OH_200px', 'VC_200px']


VISUALISE DATA

In [8]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(5, 200, 200, 3)
(5,)


Configure the dataset for performance

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(50).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Standardize the data

CREATE MODEL

In [10]:
num_classes = 6

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [19]:
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0

In [20]:
epochs = 100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs = epochs
)

Epoch 1/100
107/107 [==============================] - 1s 9ms/step - loss: 1.7918 - accuracy: 0.1697 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 2/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1925 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 3/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1772 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 4/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1394 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 5/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1578 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 6/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1693 - val_loss: 1.7918 - val_accuracy: 0.2203
Epoch 7/100
107/107 [==============================] - 1s 7ms/step - loss: 1.7918 - accuracy: 0.1517 - val_loss: 1.7918 - val_accuracy: 0.2203

In [14]:
keras_file = "model.h5"
keras.models.save_model(model, keras_file)

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmptfnxuqj_/assets


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
